In [1]:
import numpy as np
import glob
import os
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
#GPU 사용하도록 설정
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
gesture = {
    0:"A", 1:"B", 2:"C", 3:"D", 4:"E", 5:"F", 6:"G", 7:"H", 8:"I", 9:"J", 10:"K", 11:"L", 12:"M", 13:"N",
    14:"O", 15:"P", 16:"Q", 17:"R", 18:"S", 19:"T", 20:"U", 21:"V", 22:"W", 23:"X", 24:"Y", 25:"Z"
}

In [4]:
path = "./hand_dataset/*.npy"

In [5]:
file_list = glob.glob(path)
file_list

['./hand_dataset\\A-1726903694.npy',
 './hand_dataset\\A-1726903766.npy',
 './hand_dataset\\A_1721720113.npy',
 './hand_dataset\\B-1726903818.npy',
 './hand_dataset\\B_1721719540.npy',
 './hand_dataset\\B_1721720254.npy',
 './hand_dataset\\C-1726903900.npy',
 './hand_dataset\\C_1721719591.npy',
 './hand_dataset\\C_1721720352.npy',
 './hand_dataset\\D-1726904014.npy',
 './hand_dataset\\D_1721719643.npy',
 './hand_dataset\\D_1721720397.npy',
 './hand_dataset\\E-1726904060.npy',
 './hand_dataset\\E_1721719690.npy',
 './hand_dataset\\E_1721720442.npy',
 './hand_dataset\\F-1726904116.npy',
 './hand_dataset\\F_1721719739.npy',
 './hand_dataset\\F_1721722482.npy',
 './hand_dataset\\G-1726904362.npy',
 './hand_dataset\\G_1721719786.npy',
 './hand_dataset\\G_1721722528.npy',
 './hand_dataset\\H-1726904414.npy',
 './hand_dataset\\H_1721719834.npy',
 './hand_dataset\\H_1721722613.npy',
 './hand_dataset\\I-1726904459.npy',
 './hand_dataset\\I_1721719998.npy',
 './hand_dataset\\I_1721722659.npy',
 

In [6]:
data_list = []
for file in file_list:
    print("file=", file)
    data = np.load(file)
    print("data.shape=", data[:540, :].shape)
    data_list.extend(data[:540, :])

file= ./hand_dataset\A-1726903694.npy
data.shape= (540, 79)
file= ./hand_dataset\A-1726903766.npy
data.shape= (540, 79)
file= ./hand_dataset\A_1721720113.npy
data.shape= (540, 79)
file= ./hand_dataset\B-1726903818.npy
data.shape= (540, 79)
file= ./hand_dataset\B_1721719540.npy
data.shape= (540, 79)
file= ./hand_dataset\B_1721720254.npy
data.shape= (540, 79)
file= ./hand_dataset\C-1726903900.npy
data.shape= (540, 79)
file= ./hand_dataset\C_1721719591.npy
data.shape= (540, 79)
file= ./hand_dataset\C_1721720352.npy
data.shape= (540, 79)
file= ./hand_dataset\D-1726904014.npy
data.shape= (540, 79)
file= ./hand_dataset\D_1721719643.npy
data.shape= (540, 79)
file= ./hand_dataset\D_1721720397.npy
data.shape= (540, 79)
file= ./hand_dataset\E-1726904060.npy
data.shape= (540, 79)
file= ./hand_dataset\E_1721719690.npy
data.shape= (540, 79)
file= ./hand_dataset\E_1721720442.npy
data.shape= (540, 79)
file= ./hand_dataset\F-1726904116.npy
data.shape= (540, 79)
file= ./hand_dataset\F_1721719739.npy
da

In [7]:
len(data_list)

38880

In [9]:
all_data = np.array(data_list, dtype=np.float32)
all_data

array([[ 4.9234158e-01,  9.6115118e-01, -5.2998939e-07, ...,
         4.3858547e+01,  1.1889682e+02,  0.0000000e+00],
       [ 5.1829427e-01,  7.6567298e-01, -4.9394811e-07, ...,
         5.5679295e+01,  5.2032928e+01,  0.0000000e+00],
       [ 5.3892606e-01,  7.2008771e-01, -1.8606367e-07, ...,
         1.4431424e+02,  6.3804428e+01,  0.0000000e+00],
       ...,
       [ 4.7306544e-01,  6.2992150e-01,  4.6825051e-08, ...,
         8.9414604e+01,  6.6004005e+01,  2.5000000e+01],
       [ 4.8888662e-01,  6.3018769e-01,  1.2027233e-07, ...,
         6.9013115e+01,  7.8130768e+01,  2.5000000e+01],
       [ 5.0208497e-01,  6.2503552e-01,  8.3170420e-08, ...,
         6.7569412e+01,  7.8076332e+01,  2.5000000e+01]], dtype=float32)

In [10]:
all_data.shape

(38880, 79)

In [11]:
all_data_lstm = all_data.reshape(-1, 5, 79)
all_data_lstm.shape

(7776, 5, 79)

In [12]:
X_data = all_data_lstm[:,:,:-1]
X_data

array([[[ 4.92341578e-01,  9.61151183e-01, -5.29989393e-07, ...,
          1.30777252e+02,  4.38585472e+01,  1.18896820e+02],
        [ 5.18294275e-01,  7.65672982e-01, -4.93948107e-07, ...,
          1.30593277e+02,  5.56792946e+01,  5.20329285e+01],
        [ 5.38926065e-01,  7.20087707e-01, -1.86063673e-07, ...,
          3.25502052e+01,  1.44314240e+02,  6.38044281e+01],
        [ 5.44440687e-01,  6.53261304e-01, -1.87380195e-07, ...,
          1.78087082e+01,  1.52062119e+02,  3.47539673e+01],
        [ 5.48921227e-01,  5.91867328e-01, -2.60911577e-07, ...,
          1.73126469e+01,  1.63197250e+02,  3.34668159e+01]],

       [[ 5.46441376e-01,  5.39192617e-01, -3.39433143e-07, ...,
          2.06088524e+01,  1.67759689e+02,  2.21394196e+01],
        [ 5.39980531e-01,  5.00345349e-01, -3.84503295e-07, ...,
          2.16697292e+01,  1.67811707e+02,  1.79285316e+01],
        [ 5.39938986e-01,  4.80360836e-01, -3.85743618e-07, ...,
          2.14865837e+01,  1.67894333e+02,  1.80005

In [13]:
labels = all_data_lstm[:,0,-1]
labels

array([ 0.,  0.,  0., ..., 25., 25., 25.], dtype=float32)

In [14]:
y_data = to_categorical(labels, num_classes=len(gesture))
y_data

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.1)

In [16]:
X_train

array([[[ 5.45611024e-01,  3.50237906e-01,  7.60441253e-07, ...,
          6.62233276e+01,  1.05251236e+02,  4.47323532e+01],
        [ 5.08223236e-01,  3.73017728e-01,  6.04373838e-07, ...,
          6.38233681e+01,  7.50317230e+01,  4.57805405e+01],
        [ 5.00979125e-01,  4.21817780e-01,  5.59251077e-07, ...,
          7.29456482e+01,  6.79581604e+01,  4.55314026e+01],
        [ 5.04022837e-01,  4.40775305e-01,  5.93977518e-07, ...,
          6.44964905e+01,  6.86083450e+01,  4.58035583e+01],
        [ 5.06257892e-01,  4.54897225e-01,  6.16927196e-07, ...,
          7.22441177e+01,  6.94507523e+01,  4.38472862e+01]],

       [[ 4.33888882e-01,  4.93212998e-01, -3.02041798e-07, ...,
          2.37246914e+01,  1.34024643e+02,  2.88025665e+01],
        [ 4.26506907e-01,  4.92526352e-01, -3.10838601e-07, ...,
          2.34994965e+01,  1.29867538e+02,  3.06327171e+01],
        [ 4.17606711e-01,  4.92050052e-01, -3.21365349e-07, ...,
          2.39757729e+01,  1.27471832e+02,  2.90656

In [17]:
y_train[0:5]

array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [31]:
model = Sequential()

In [32]:
model.add(LSTM(256, input_shape=(5, 78)))

In [33]:
model.add(Flatten())

In [34]:
model.add(Dense(128, activation='relu'))

In [35]:
model.add(Dropout(0.5))

In [36]:
model.add(Dense(64, activation='relu'))

In [37]:
model.add(Dropout(0.5))

In [38]:
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))

In [39]:
model.add(Dense(len(gesture), activation='softmax'))

In [40]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [41]:
cb_checkpoint = ModelCheckpoint(filepath='./hand_lstm_train_result/', monitor='val_acc', save_best_only=True)

In [42]:
earlystopping = EarlyStopping(monitor="val_loss", patience=30)

In [43]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, callbacks=[cb_checkpoint, earlystopping])

Epoch 1/100
208/219 [===========================>..] - ETA: 0s - loss: 2.9665 - acc: 0.1340

INFO:tensorflow:Assets written to: ./hand_lstm_train_result\assets


INFO:tensorflow:Assets written to: ./hand_lstm_train_result\assets


219/219 [==============================] - 11s 19ms/step - loss: 2.9476 - acc: 0.1389 - val_loss: 2.3531 - val_acc: 0.3175
Epoch 2/100
211/219 [===========================>..] - ETA: 0s - loss: 2.3556 - acc: 0.2475

INFO:tensorflow:Assets written to: ./hand_lstm_train_result\assets


...

INFO:tensorflow:Assets written to: ./hand_lstm_train_result\assets


219/219 [==============================] - 3s 13ms/step - loss: 0.5153 - acc: 0.8252 - val_loss: 0.2526 - val_acc: 0.9293
Epoch 94/100
219/219 [==============================] - 1s 4ms/step - loss: 0.4845 - acc: 0.8417 - val_loss: 0.2613 - val_acc: 0.9165
Epoch 95/100
219/219 [==============================] - 1s 3ms/step - loss: 0.4770 - acc: 0.8418 - val_loss: 0.2561 - val_acc: 0.9203
Epoch 96/100
219/219 [==============================] - 1s 3ms/step - loss: 0.5095 - acc: 0.8395 - val_loss: 0.2778 - val_acc: 0.9075
Epoch 97/100
216/219 [============================>.] - ETA: 0s - loss: 0.4698 - acc: 0.8436

INFO:tensorflow:Assets written to: ./hand_lstm_train_result\assets


INFO:tensorflow:Assets written to: ./hand_lstm_train_result\assets


219/219 [==============================] - 3s 13ms/step - loss: 0.4693 - acc: 0.8440 - val_loss: 0.2299 - val_acc: 0.9319
Epoch 98/100
219/219 [==============================] - 1s 5ms/step - loss: 0.4818 - acc: 0.8391 - val_loss: 0.2756 - val_acc: 0.9216
Epoch 99/100
219/219 [==============================] - 1s 4ms/step - loss: 0.5290 - acc: 0.8228 - val_loss: 0.2694 - val_acc: 0.9139
Epoch 100/100
219/219 [==============================] - 1s 4ms/step - loss: 0.5021 - acc: 0.8378 - val_loss: 0.2984 - val_acc: 0.9165
